In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-227344
Azure region: eastus2
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-227344


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpuCluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
cpu_cluster = ComputeTarget.create(ws,cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C":uniform(0.05,0.1),
    "max_iter":choice(16,32,64,128)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory="./",script="train.py", compute_target=cluster_name, environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name="Accuracy",
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=20
)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
experiment= Experiment(ws, "expierment_0")
run = experiment.submit(config= hyperdrive_config, show_output=True)

In [7]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [30]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_model = run.get_best_run_by_primary_metric()
print(best_model)

Run(Experiment: expierment_0,
Id: HD_46463bae-bce5-428f-8491-bdac7d23f61f_13,
Type: azureml.scriptrun,
Status: Completed)


In [24]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [25]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [46]:
x['age']

0        57
1        55
2        33
3        36
4        27
         ..
32945    56
32946    37
32947    26
32948    31
32949    39
Name: age, Length: 32950, dtype: int64

In [47]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=x,
    label_column_name='age',
    n_cross_validations=5)

In [48]:
# Submit your automl run

### YOUR CODE HERE ###
experiment= Experiment(ws, "auto_ml_experiment")
run = experiment.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
auto_ml_experiment,AutoML_88c1fa82-e02b-4489-8157-edb053111005,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2023/03/07 17:03:59 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-03-07:17:36:28,455 INFO     [explanation_client.py:334] Using default datastore for uploads
2023-03-07:17:38:22,915 WARNING  [explanation_utils.py:92] Cannot serialize JSON possibly due to timestamp, retrying
2023-03-07:17:38:23,269 WARNING  [explanation_utils.py:92] Cannot serialize JSON possibly due to timestamp, retrying


In [49]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_model = run.
print(best_model)

AttributeError: 'AutoMLRun' object has no attribute 'get_best_run_by_primary_metric'